In [1]:
# IMPORTANT: Ensure this file is not named "torch.py" or "transformers.py" to avoid circular import conflicts.

import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle
from tensorflow import keras

    
df = pd.read_csv("MetroPT3(AirCompressor).csv")
print("File successfully read.")

# Define alert conditions
condition_1 = (df['TP2'] > -0.1) & (df['TP2'] < 0.2)  # Pressure imbalance
condition_2 = df['TP3'] != df['Reservoirs']            # Reservoir mismatch
condition_3 = (df['H1'] < 7)                            # Low H1
condition_4 = (df['DV_pressure'] <= 0) | (df['DV_pressure'] >= 0.2)  # Diverter valve pressure issue
condition_5 = (df['Oil_temperature'] < 30) | (df['Oil_temperature'] > 90)  # Oil overheating
condition_6 = df['DV_eletric'] == 0                     # Electrical failure

# Assign alerts
df['target'] = 0  # Default: Normal
df.loc[condition_6, 'target'] = 1  # Electrical failure
df.loc[(df['DV_eletric'] == 1) & (condition_1 | condition_2 | condition_3 | condition_4 | condition_5), 'target'] = 1

# Add Alert Column
df['Alert'] = np.where(df['target'] == 1, "ALERT", "NORMAL")

    # -------------------------------------------------------
    # AI-BASED PREVENTIVE MEASURES GENERATOR AND DISCRIMINATOR
    # -------------------------------------------------------

    # 1. Generator: Using a transformer pipeline to generate preventive measures.
    # We're using 'gpt2' here for demonstration.
generator = pipeline("text-generation", model="gpt2")

def generate_preventive_measures(issue):
    """
    Given an issue description, use the AI generator to suggest preventive measures.
    """
    prompt = f"Given the issue: '{issue}', provide three concise and practical preventive measures."
    response = generator(prompt, max_length=100, num_return_sequences=1)
    # Extract generated text from the response (adjust if needed based on model output)
    generated_text = response[0]['generated_text']
    return generated_text.strip()

    # 2. Discriminator: Load a pre-trained classifier and vectorizer (assumed to be trained on historical data)
try:
    with open("discriminator_model.pkl", "rb") as f:
        discriminator = pickle.load(f)
    with open("vectorizer.pkl", "rb") as f:
        vectorizer = pickle.load(f)
except Exception as e:
    print("Discriminator models not found. Please ensure 'discriminator_model.pkl' and 'vectorizer.pkl' exist.")
    # For demonstration, create dummy models that always approve the measure
    discriminator = None
    vectorizer = None

def validate_measure(measure):
    """
    Validate the generated measure using the discriminator.
    Returns True if approved, False otherwise.
    If no discriminator is loaded, default to approve.
    """
    if discriminator is None or vectorizer is None:
        return True  # Approve by default in demo mode
    X_test = vectorizer.transform([measure])
    prediction = discriminator.predict(X_test)
    return prediction[0] == 1  # Assuming label 1 means 'Approved'

# -------------------------------------------------------
# Modify get_alert_details to use the generator and discriminator
def get_alert_details(row):
    issues = []
    measures = []

    # Pressure Imbalance Issue
    if row['TP2'] <= -0.1 or row['TP2'] >= 0.2:
        issue_text = "Pressure Imbalance: TP2 sensor might be faulty or there is a leakage."
        gen_measure = generate_preventive_measures(issue_text)
        if validate_measure(gen_measure):
            issues.append("🔴 " + issue_text)
            measures.append("✔️ " + gen_measure)

    # Reservoir Pressure Mismatch Issue
    if row['TP3'] != row['Reservoirs']:
        issue_text = "Reservoir Pressure Mismatch: Reservoirs and TP3 values do not match."
        gen_measure = generate_preventive_measures(issue_text)
        if validate_measure(gen_measure):
            issues.append("⚠️ " + issue_text)
            measures.append("✔️ " + gen_measure)

    # Low H1 Issue
    if row['H1'] < 7:
        issue_text = "Low H1 Value: Air compressor humidity level is below normal."
        gen_measure = generate_preventive_measures(issue_text)
        if validate_measure(gen_measure):
            issues.append("🔧 " + issue_text)
            measures.append("✔️ " + gen_measure)

    # Diverter Valve Pressure Issue
    if row['DV_pressure'] <= 0 or row['DV_pressure'] >= 0.2:
        issue_text = "Diverter Valve Pressure Issue: DV pressure is outside the safe range."
        gen_measure = generate_preventive_measures(issue_text)
        if validate_measure(gen_measure):
            issues.append("🛑 " + issue_text)
            measures.append("✔️ " + gen_measure)

    # Oil Temperature Abnormality Issue
    if row['Oil_temperature'] < 30 or row['Oil_temperature'] > 90:
        issue_text = "Oil Temperature Abnormal: Possible overheating or lubrication issue."
        gen_measure = generate_preventive_measures(issue_text)
        if validate_measure(gen_measure):
            issues.append("🔥 " + issue_text)
            measures.append("✔️ " + gen_measure)

    # Electrical Failure Issue
    if row['DV_eletric'] == 0:
        issue_text = "Electrical Failure: No power detected in the system."
        gen_measure = generate_preventive_measures(issue_text)
        if validate_measure(gen_measure):
            issues.append("⚡ " + issue_text)
            measures.append("✔️ " + gen_measure)

    return issues, measures

# -------------------------------------------------------
# Sample alert rows for pop-up alerts
alert_rows = df[df['Alert'] == "ALERT"]
if not alert_rows.empty:
    sample_1 = alert_rows.sample(n=1, random_state=42)  # First faulty equipment
    remaining_alerts = alert_rows.drop(sample_1.index)  # Remove first sampled equipment
    sample_2 = remaining_alerts.sample(n=1, random_state=24) if not remaining_alerts.empty else pd.DataFrame()  # Second faulty equipment

    sampled_alerts = pd.concat([sample_1, sample_2]) if not sample_2.empty else sample_1  # Combine both samples
else:
    sampled_alerts = pd.DataFrame()

# Function to display a single pop-up alert
def show_alert():
    if not sampled_alerts.empty:
        alert_texts = []
        for index, row in sampled_alerts.iterrows():
            issues, measures = get_alert_details(row)
            alert_texts.append(
                f"🚨 ALERT: Equipment {index} requires immediate attention!\n\n"
                f"Issue(s):\n" + "\n".join(issues) + "\n\n"
                "⚠️ Preventive Measures:\n" + "\n".join(measures)
            )
        alert_msg = "\n\n---\n\n".join(alert_texts)

        root = tk.Tk()
        root.withdraw()  # Hide the root window
        root.attributes('-topmost', True)  # Keep pop-up on top
        messagebox.showwarning("Equipment Alert", alert_msg)  # Show warning pop-up
        root.destroy()  # Close pop-up after displaying

# Show alert pop-up
show_alert()

# Show well-functioning equipment details in console
normal_rows = df[df['Alert'] == "NORMAL"]

if len(normal_rows) >= 3:
    normal_rows = normal_rows.sample(n=3, random_state=42)
elif not normal_rows.empty:  # If fewer than 3 normal rows exist, sample all available
    normal_rows = normal_rows.sample(n=len(normal_rows), random_state=42)
else:
    normal_rows = pd.DataFrame()  # No normal equipment available

if not normal_rows.empty:
    print("\n--- Well-Functioning Equipment ---")
    for index, row in normal_rows.iterrows():
        print(f"✅ Equipment {index} is functioning well.")
        print(f"Details: TP2={row['TP2']}, TP3={row['TP3']}, Reservoirs={row['Reservoirs']}, "
              f"H1={row['H1']}, DV_pressure={row['DV_pressure']}, Oil_temperature={row['Oil_temperature']}, "
              f"DV_eletric={row['DV_eletric']}")
        print("-" * 60)

# Save sampled alert details to a CSV file
sampled_alerts.to_csv("sampled_alerts.csv", index=False)
print("\nSampled alert details saved to 'sampled_alerts.csv'.")


c:\Users\bhara\anaconda3\envs\miniProject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


File successfully read.


c:\Users\bhara\anaconda3\envs\miniProject\lib\site-packages\huggingface_hub-0.28.1-py3.8.egg\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
c:\Users\bhara\anaconda3\envs\miniProject\lib\site-packages\huggingface_hub-0.28.1-py3.8.egg\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bhara\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activa

Discriminator models not found. Please ensure 'discriminator_model.pkl' and 'vectorizer.pkl' exist.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Well-Functioning Equipment ---
✅ Equipment 597437 is functioning well.
Details: TP2=0.23, TP3=9.97, Reservoirs=9.97, H1=8.15, DV_pressure=0.168, Oil_temperature=60.775, DV_eletric=1
------------------------------------------------------------
✅ Equipment 332221 is functioning well.
Details: TP2=0.304, TP3=10.024, Reservoirs=10.024, H1=9.804, DV_pressure=0.118, Oil_temperature=64.1, DV_eletric=1
------------------------------------------------------------

Sampled alert details saved to 'sampled_alerts.csv'.
